In [ ]:
def strategy_6(data):
    def calculate_heiken_ashi(df):
        ha_close = (df['open'] + df['high'] + df['low'] + df['close']) / 4
        ha_open = (df['open'].shift(1) + df['close'].shift(1)) / 2
        ha_high = df[['high', 'open', 'close']].max(axis=1)
        ha_low = df[['low', 'open', 'close']].min(axis=1)

        df['HA_Close_6'] = ha_close
        df['HA_Open_6'] = ha_open
        df['HA_High_6'] = ha_high
        df['HA_Low_6'] = ha_low
        df['HA_Color_6'] = 'green'
        df.loc[df['HA_Close_6'] < df['HA_Open_6'], 'HA_Color_6'] = 'red'

        return df

    data = calculate_heiken_ashi(data)

    n = 14
    data['EMA_6'] = ta.EMA(data['close'], timeperiod=14)
    data['ATR_6'] = ta.ATR(data['high'], data['low'], data['close'], timeperiod=13)
    data['Upper_Line_6'] = data['EMA_6'] + 2 * data['ATR_6']
    data['Lower_Line_6'] = data['EMA_6'] - 2 * data['ATR_6']
    data = data.drop(columns=['ATR_6'])

    margin = 0.35
    data['body_6'] = data['high'] - data['low']
    data['upper_d_6'] = data['high'] - data['open']
    data['lower_d_6'] = data['close'] - data['low']
    data['upper_u_6'] = data['high'] - data['close']
    data['lower_u_6'] = data['open'] - data['low']

    data['piercing_6'] = 0

    for i in range(1, len(data)):
        if (data['open'][i-1] > data['close'][i-1] and data['upper_d_6'][i-1] < margin * data['body_6'][i-1] and data['lower_d_6'][i-1] < margin * data['body_6'][i-1]):
            if (data['close'][i] > data['open'][i] and data['open'][i] < data['close'][i-1] and data['upper_u_6'][i] < margin * data['body_6'][i] and data['lower_u_6'][i] < margin * data['body_6'][i]):
                data['piercing_6'][i] = 1
        elif (data['open'][i-1] < data['close'][i-1] and data['upper_d_6'][i-1] < margin * data['body_6'][i-1] and data['lower_d_6'][i-1] < margin * data['body_6'][i-1]):
            if (data['close'][i] < data['open'][i] and data['open'][i] > data['close'][i-1] and data['upper_u_6'][i] < margin * data['body_6'][i] and data['lower_u_6'][i] < margin * data['body_6'][i]):
                data['piercing_6'][i] = -1

    data['macd_6'] = data['high'].ewm(span=12, adjust=False).mean() - data['high'].ewm(span=28, adjust=False).mean()
    data['signal_macd_6'] = data['macd_6'].ewm(span=9, adjust=False).mean()

    data['IBS_6'] = (data['close'] - data['low']) / (data['high'] - data['low'])
    data['weight_avg_6'] = data['IBS_6'].ewm(span=6, adjust=False).mean()

    def rsi(df, period=16):
        df['rsi_6'] = ta.RSI(df['close'], period)
        return df
    
    data = rsi(data)

    data['signals'] = 0
    for i in range(len(data)):
        if (data['close'][i] > data['Upper_Line_6'][i] and (data['rsi_6'][i] > 71.9 or (data['weight_avg_6'][i] > 0.7 and data['macd_6'][i] > data['signal_macd_6'][i])) and data['piercing_6'][i] == 0):
            if data['HA_Color_6'][i] == 'green':
                data['signals'][i] = 1
        elif (data['close'][i] < data['Lower_Line_6'][i] and (data['rsi_6'][i] < 28.1 or (data['weight_avg_6'][i] < 0.3 and data['macd_6'][i] < data['signal_macd_6'][i])) and data['piercing_6'][i] == 0):
            if data['HA_Color_6'][i] == 'red':
                data['signals'][i] = -1

    return data
